In [76]:
pip install autots


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [78]:
from autots import AutoTS, load_daily
from sklearn.preprocessing import MinMaxScaler
import pandas as pd


In [99]:
import pandas as pd
ts_ms = pd.date_range(start = '2023-03-10 23:13:20',     # 날짜 범위 시작
                     end = None,                # 날짜 범위 끝
                     periods = 19304,               # 생성할 Timestamp 개수
                     freq = 'T',               # 시간 간격(MS : 월의 시작일)
                     tz = 'Asia/Seoul')         # 시간대(timezone)
print(ts_ms)

DatetimeIndex(['2023-03-10 23:13:20+09:00', '2023-03-10 23:14:20+09:00',
               '2023-03-10 23:15:20+09:00', '2023-03-10 23:16:20+09:00',
               '2023-03-10 23:17:20+09:00', '2023-03-10 23:18:20+09:00',
               '2023-03-10 23:19:20+09:00', '2023-03-10 23:20:20+09:00',
               '2023-03-10 23:21:20+09:00', '2023-03-10 23:22:20+09:00',
               ...
               '2023-03-24 08:47:20+09:00', '2023-03-24 08:48:20+09:00',
               '2023-03-24 08:49:20+09:00', '2023-03-24 08:50:20+09:00',
               '2023-03-24 08:51:20+09:00', '2023-03-24 08:52:20+09:00',
               '2023-03-24 08:53:20+09:00', '2023-03-24 08:54:20+09:00',
               '2023-03-24 08:55:20+09:00', '2023-03-24 08:56:20+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', length=19304, freq='T')


In [101]:
long = False
df = pd.read_csv('/Users/sunflower/Korea/실험환경/python/data/cpu_memory_usage_v1.csv', header=0)
df = df.loc[df.pod_name == 'drive-az2-prd-69cbb999bc-lpr67']
df = df.loc[:,['index','time','cpu_usage_nanocores', 'memory_working_set_bytes']]
df = df.dropna()


ms = MinMaxScaler()

df[['cpu_usage_nanocores','memory_working_set_bytes']] = ms.fit_transform(df[['cpu_usage_nanocores','memory_working_set_bytes']])
df['time'] = ts_ms
df.set_index('time', inplace=True)
df

,index,cpu_usage_nanocores,memory_working_set_bytes
time,,,
2023-03-10 23:13:20+09:00,2,0.411576,0.044959
2023-03-10 23:14:20+09:00,18,0.481051,0.044911
2023-03-10 23:15:20+09:00,31,0.500366,0.049663
2023-03-10 23:16:20+09:00,39,0.441449,0.049532
2023-03-10 23:17:20+09:00,52,0.681631,0.050024
...,...,...,...
2023-03-24 08:52:20+09:00,231587,0.031644,0.066024
2023-03-24 08:53:20+09:00,231589,0.024549,0.066039
2023-03-24 08:54:20+09:00,231605,0.036685,0.066039


In [102]:

model = AutoTS(
    forecast_length=21,
    frequency='infer',
    prediction_interval=0.9,
    ensemble=None,
    model_list="fast",  # "superfast", "default", "fast_parallel"
    transformer_list="fast",  # "superfast",
    drop_most_recent=1,
    max_generations=4,
    num_validations=2,
    validation_method="backwards"
)

In [103]:

model = model.fit(
    df,
    date_col='time' if long else None,
    value_col='memory_working_set_bytes' if long else None,
    id_col='index' if long else None,
)


Inferred frequency is: T
Model Number: 1 with model AverageValueNaive in generation 0 of 4
Model Number: 2 with model AverageValueNaive in generation 0 of 4
Model Number: 3 with model AverageValueNaive in generation 0 of 4
Model Number: 4 with model DatepartRegression in generation 0 of 4
Model Number: 5 with model DatepartRegression in generation 0 of 4


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model Number: 6 with model DatepartRegression in generation 0 of 4


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Model Number: 7 with model DatepartRegression in generation 0 of 4
Epoch 1/50


...

/opt/homebrew/anaconda3/lib/python3.10/site-packages/autots/tools/holiday.py:40: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  USFederalHolidayCalendar()
/opt/homebrew/anaconda3/lib/python3.10/site-packages/autots/tools/holiday.py:40: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  USFederalHolidayCalendar()


Model Number: 62 with model SeasonalityMotif in generation 0 of 4
Model Number: 63 with model MetricMotif in generation 0 of 4
Model Number: 64 with model ETS in generation 0 of 4
Template Eval Error: Exception('Transformer DatepartRegression failed on fit') in model 64 in generation 0: ETS
Model Number: 65 with model VECM in generation 0 of 4
Model Number: 66 with model MAR in generation 0 of 4
Template Eval Error: ValueError('Shape of passed values is (7, 3), indices imply (21, 3)') in model 66 in generation 0: MAR
Model Number: 67 with model GLS in generation 0 of 4
Model Number: 68 with model MAR in generation 0 of 4
Model Number: 69 with model ETS in generation 0 of 4
Model Number: 70 with model VECM in generation 0 of 4
Template Eval Error: ValueError("regression_type='User' but no future_regressor supplied") in model 70 in generation 0: VECM
Model Number: 71 with model SectionalMotif in generation 0 of 4
Model Number: 72 with model ETS in generation 0 of 4
ETS error ValueError('

/opt/homebrew/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:48: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:550: RuntimeWarning: invalid value encountered in divide
  avg = np.multiply(a, wgt,


Model Number: 84 with model SectionalMotif in generation 0 of 4
Model Number: 85 with model VAR in generation 0 of 4


/opt/homebrew/anaconda3/lib/python3.10/site-packages/autots/tools/holiday.py:40: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  USFederalHolidayCalendar()
/opt/homebrew/anaconda3/lib/python3.10/site-packages/autots/tools/holiday.py:40: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  USFederalHolidayCalendar()


In [ ]:

prediction = model.predict()

In [ ]:

# plot a sample
prediction.plot(model.df_wide_numeric,
                series=model.df_wide_numeric.columns[0],
                start_date="2019-01-01")

In [ ]:

# Print the details of the best model
print(model)

# point forecasts dataframe
forecasts_df = prediction.forecast
# upper and lower forecasts
forecasts_up, forecasts_low = prediction.upper_forecast, prediction.lower_forecast

In [ ]:

# accuracy of all tried model results
model_results = model.results()
# and aggregated from cross validation
validation_results = model.results("validation")